In [ ]:
import os
import numpy as np
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]  # 找到所有mat文件
file_list = sorted(file_list)  # 按文件名排序，比如 dev0.mat, dev1.mat, dev2.mat

data = []  # 用来存储每个mat文件中的 'data_Ineed'（转置后的）

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)
    if 'data_Ineed' in mat_contents:
        data_ineed = mat_contents['data_Ineed'].T  # 加上 .T 转置！
        
        # 数据格式转化，按 320 信号一块进行处理
        num_signals, num_samples = data_ineed.shape
        
        # 确保每 320 个信号作为一个大块进行处理
        block_size = 320
        num_blocks = num_signals // block_size  # 计算可以完整分成多少块

        # 舍去多余的信号
        data_ineed = data_ineed[:num_blocks * block_size, :]
        
        # 将信号数据重构：每 320 个信号为一个大块
        reshaped_data = data_ineed.reshape((block_size, num_blocks * num_samples)).T  # 转置回原来的格式

        data.append(reshaped_data)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (处理后): {d.shape}")


In [ ]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = -20  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

print(f"=== 开始添加噪声 (SNR={SNR_dB}dB) ===")

for i, d in enumerate(data):
    # 检查信号类型（实数/复数）
    is_complex = np.iscomplexobj(d)
    
    if is_complex:
        print(f"文件{i}: 检测到复数信号，shape={d.shape}")
        
        # 计算复数信号功率
        signal_power = np.mean(np.abs(d) ** 2)
        
        # 计算噪声功率
        noise_power = signal_power / (10 ** (SNR_dB / 10))
        
        # 复数噪声：实部和虚部各占一半功率
        noise_std = np.sqrt(noise_power / 2)
        
        # 生成独立的实部和虚部噪声
        real_noise = np.random.normal(0, noise_std, d.shape)
        imag_noise = np.random.normal(0, noise_std, d.shape) * 1j
        
        # 合成复数噪声并添加到信号
        complex_noise = real_noise + imag_noise
        noisy_signal = d + complex_noise
        
    else:
        print(f"文件{i}: 实数信号，shape={d.shape}")
        
        # 计算信号的标准差
        signal_std = np.std(d)
        
        # 计算噪声的标准差，根据信噪比（SNR）
        noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
        
        # 生成高斯噪声并加到原信号上
        noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
        noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

print("\n✅ 所有文件噪声添加完成！")

# 详细的验证和对比
print("\n" + "="*50)
print("噪声添加效果验证")
print("="*50)

for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    actual_noise = noisy - orig
    
    if np.iscomplexobj(orig):
        # 复数信号验证
        signal_power = np.mean(np.abs(orig) ** 2)
        noise_power = np.mean(np.abs(actual_noise) ** 2)
        actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
        
        print(f"\n文件{i} - 复数信号验证:")
        print(f"  📊 目标SNR: {SNR_dB}dB, 实际SNR: {actual_snr_db:.2f}dB")
        print(f"  ⚡ 信号功率: {signal_power:.6f}")
        print(f"  🔊 噪声功率: {noise_power:.6f}")
        print(f"  📏 实部噪声标准差: {np.std(actual_noise.real):.6f}")
        print(f"  📏 虚部噪声标准差: {np.std(actual_noise.imag):.6f}")
        
    else:
        # 实数信号验证
        signal_power = np.var(orig)
        noise_power = np.var(actual_noise)
        actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
        
        print(f"\n文件{i} - 实数信号验证:")
        print(f"  📊 目标SNR: {SNR_dB}dB, 实际SNR: {actual_snr_db:.2f}dB")
        print(f"  ⚡ 信号功率: {signal_power:.6f}")
        print(f"  🔊 噪声功率: {noise_power:.6f}")
        print(f"  📏 噪声标准差: {np.std(actual_noise):.6f}")

# 打印检查加噪声后的数据样本
print("\n" + "="*50)
print("数据样本详细对比")
print("="*50)

for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    print(f"\n📁 文件{i} - 前5个采样点对比:")
    
    if np.iscomplexobj(orig):
        # 复数信号样本对比
        print("  索引 | 原始信号 (实部+虚部j) | 加噪信号 (实部+虚部j) | 噪声 (实部+虚部j)")
        print("  " + "-" * 80)
        for j in range(min(5, orig.shape[0])):
            orig_val = orig[j, 0] if orig.ndim > 1 else orig[j]
            noisy_val = noisy[j, 0] if noisy.ndim > 1 else noisy[j]
            noise_val = actual_noise[j, 0] if actual_noise.ndim > 1 else actual_noise[j]
            
            print(f"  {j:2d}   | {orig_val.real:7.4f}{orig_val.imag:+.4f}j | "
                  f"{noisy_val.real:7.4f}{noisy_val.imag:+.4f}j | "
                  f"{noise_val.real:7.4f}{noise_val.imag:+.4f}j")
    else:
        # 实数信号样本对比
        print("  索引 | 原始信号 | 加噪信号 | 噪声")
        print("  " + "-" * 50)
        for j in range(min(5, orig.shape[0])):
            orig_val = orig[j, 0] if orig.ndim > 1 else orig[j]
            noisy_val = noisy[j, 0] if noisy.ndim > 1 else noisy[j]
            noise_val = actual_noise[j, 0] if actual_noise.ndim > 1 else actual_noise[j]
            
            print(f"  {j:2d}   | {orig_val:8.4f} | {noisy_val:8.4f} | {noise_val:8.4f}")

# 统计信息总结
print("\n" + "="*50)
print("添加噪声统计总结")
print("="*50)

total_files = len(data)
complex_files = sum(1 for d in data if np.iscomplexobj(d))
real_files = total_files - complex_files

print(f"📂 总文件数: {total_files}")
print(f"🔷 复数信号文件: {complex_files}")
print(f"🔶 实数信号文件: {real_files}")

# 检查SNR控制精度
snr_errors = []
for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    actual_noise = noisy - orig
    if np.iscomplexobj(orig):
        signal_power = np.mean(np.abs(orig) ** 2)
        noise_power = np.mean(np.abs(actual_noise) ** 2)
    else:
        signal_power = np.var(orig)
        noise_power = np.var(actual_noise)
    
    actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
    snr_error = abs(actual_snr_db - SNR_dB)
    snr_errors.append(snr_error)

avg_snr_error = np.mean(snr_errors)
max_snr_error = np.max(snr_errors)

print(f"🎯 SNR控制精度:")
print(f"  平均误差: {avg_snr_error:.4f} dB")
print(f"  最大误差: {max_snr_error:.4f} dB")
print(f"  {'✅ 优秀' if avg_snr_error < 0.1 else '⚠️ 良好' if avg_snr_error < 0.5 else '❌ 需要检查'}")

print(f"\n🎉 噪声添加完成！所有文件已处理并验证。")

In [ ]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    v = v/3.6 # 换算成m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，(km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from datetime import datetime
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# =================== 设备检查 ===================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =================== 数据准备 ===================
# 假设 data_with_doppler 是你的列表，每个元素 shape = (样本数, 320)
all_data = []
all_labels = []

for device_idx, device_signals in enumerate(data_with_doppler):
    device_labels = np.full((device_signals.shape[0],), device_idx)
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

all_data = torch.cat(all_data, dim=0)
all_labels = torch.cat(all_labels, dim=0)

dataset = TensorDataset(all_data, all_labels)

# ------------------- 划分训练/验证/测试集 -------------------
test_ratio = 0.2
val_ratio = 0.2

test_size = int(len(dataset) * test_ratio)
train_val_size = len(dataset) - test_size

train_val_dataset, test_dataset = random_split(dataset, [train_val_size, test_size])

train_size = int(train_val_size * (1 - val_ratio))
val_size = train_val_size - train_size

train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# ==== 数据增强：对训练集添加高斯噪声 ====
def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)

# =================== Transformer 模型定义 ===================
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        if len(x.shape) == 3:
            x = x[:, -1, :]
        x = self.fc(x)
        return x

# =================== 随机搜索参数空间 ===================
param_space = {
    "num_heads": [2, 4, 8],
    "num_layers": [1, 2, 3],
    "dropout": [0.1, 0.3, 0.5],
    "learning_rate": [1e-3, 5e-4, 1e-4],
    "weight_decay": [1e-4, 1e-5],
    "batch_size": [64, 128, 256]
}

num_classes = 20
input_dim = 320
num_trials = 100
num_epoch = 300

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "cross_random"
folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "search_results", folder_name)
os.makedirs(save_folder, exist_ok=True)
results_file = os.path.join(save_folder, "results.txt")

trial_results = []
best_result = {"val_acc": 0}

# =================== 训练+验证+测试流程 ===================
def train_and_evaluate(hparams):
    batch_size = hparams["batch_size"]
    lr = hparams["learning_rate"]
    wd = hparams["weight_decay"]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    model = SignalTransformer(
        input_dim=input_dim,
        num_heads=hparams["num_heads"],
        num_layers=hparams["num_layers"],
        num_classes=num_classes,
        dropout=hparams["dropout"]
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    patience, patience_counter = 8, 0
    best_val_loss = float("inf")

    for epoch in range(num_epoch):
        model.train()
        train_correct, train_total = 0, 0  # 增加训练集准确率统计
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 训练集准确率
            _, preds = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (preds == labels).sum().item()

        train_acc = 100 * train_correct / train_total  # 当前 epoch 的训练集准确率

        # 验证集评估
        val_loss, val_correct, val_total = 0, 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    # 测试集评估
    model.load_state_dict(best_model_state)
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (preds == labels).sum().item()

    train_acc = 100 * train_correct / train_total
    val_acc = 100 * val_correct / val_total
    test_acc = 100 * test_correct / test_total

    return {
        "train_acc": train_acc,
        "val_loss": best_val_loss / len(val_loader),
        "val_acc": val_acc,
        "test_acc": test_acc,
        "best_model_state": best_model_state
    }


# =================== 随机搜索循环 ===================
for trial in range(num_trials):
    hparams = {k: random.choice(v) for k, v in param_space.items()}
    print(f"\n=== Trial {trial+1}/{num_trials} ===")
    print(hparams)

    metrics = train_and_evaluate(hparams)

    # 只保存超参数和验证/测试准确率
    record = {**hparams,"train_acc": metrics["train_acc"],"val_acc": metrics["val_acc"],"test_acc": metrics["test_acc"]}
    trial_results.append(record)

    print(f"Train Acc: {metrics['train_acc']:.2f}%, Validation Acc: {metrics['val_acc']:.2f}%, Test Acc: {metrics['test_acc']:.2f}%")

    with open(results_file, "a") as f:
        f.write(f"Trial {trial+1}: {record}\n")

    # 保存最优模型
    if metrics["val_acc"] > best_result.get("val_acc", 0):
        best_result = {**record}  # 记录最优参数组合及对应准确率
        best_model_path = os.path.join(save_folder, "best_model_state.pth")
        torch.save(metrics["best_model_state"], best_model_path)


with open(results_file, "a") as f:
    f.write("\n=== Best Result ===\n")
    f.write(str(best_result) + "\n")

print("Random search finished.")
print("Best params:", best_result)

# =================== 混淆矩阵绘制 ===================
def plot_confusion_matrix(best_model_state, best_hparams):
    batch_size = best_hparams["batch_size"]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    model = SignalTransformer(
        input_dim=input_dim,
        num_heads=best_hparams["num_heads"],
        num_layers=best_hparams["num_layers"],
        num_classes=num_classes,
        dropout=best_hparams["dropout"]
    ).to(device)
    model.load_state_dict(best_model_state)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(num_classes))
    plt.figure(figsize=(12, 10))
    disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
    plt.title("Confusion Matrix - Best Model")
    cm_path = os.path.join(save_folder, "best_model_confusion_matrix.png")
    plt.savefig(cm_path, dpi=300)
    plt.close()
    print(f"Confusion matrix saved to: {cm_path}")

# 调用混淆矩阵绘制
plot_confusion_matrix(torch.load(best_model_path), best_result)
